In [71]:
import numpy as np
from typing import List, Tuple, Callable
import time
import matplotlib.pyplot as plt

In [72]:
# Dữ liệu đầu vào
A = np.array([[1, 2],
              [-1, 3],
              [0.5, -2]])
b = np.array([0.1, 0.2, -0.1])

# x0 = np.random.uniform(low=-5.0, high=5.0, size=2)
x0 = np.array([0.0, 0.0])

In [73]:
# Cài đặt hàm f(x) đề bài yêu cầu và gradient của hàm f(x)
def f(x: np.ndarray) -> float:
    return np.log(np.sum(np.exp(A @ x + b)))

def grad_f(x: np.ndarray) -> np.ndarray:
    exps = np.exp(A @ x + b)
    return (exps @ A) / np.sum(exps)

def phi_k(t, xk, dk):
    return f(xk - t * dk)

# Phương pháp Parabol

In [74]:
def parabolic_interpolation_step(x1: float, x2: float, x3: float, f1: float, f2: float, f3: float) -> float:
    num = (x2 - x1) ** 2 * (f2 - f3) - (x2 - x3) ** 2 * (f2 - f1)
    den = (x2 - x1) * (f2 - f3) - (x2 - x3) * (f2 - f1)

    if abs(den) < 1e-12:
        return 0.5 * (x1 + x3)
    
    x_min = x2 - 0.5 * num / den

    return x_min

def parabol_line_search(phi: Callable, a: float = 0.0, b: float = 1.0, c: float = 2.0, tol: float = 1e-5, max_iter: int = 100) -> Tuple[float, int]:
    function_calls = 0
    x1, x2, x3 = sorted([a, b, c])
    f1 = phi(x1); f2 = phi(x2); f3 = phi(x3)
    function_calls += 3
    
    for k in range(max_iter):
        x_new = parabolic_interpolation_step(x1, x2, x3, f1, f2, f3)
        f_new = phi(x_new)
        function_calls += 1

        if abs(x2 - x_new) < tol:
            return x_new, function_calls

        if x_new < x2:
            if f_new < f2:
                x3, f3 = x2, f2
                x2, f2 = x_new, f_new
            else:
                x1, f1 = x_new, f_new
        else:
            if f_new < f2:
                x1, f1 = x2, f2
                x2, f2 = x_new, f_new
            else:
                x3, f3 = x_new, f_new
    
    return x2, function_calls

# Phương pháp Brent

In [75]:
def brent_line_search(phi: Callable, a: float = 0.0, b: float = 1.0, tol: float = 1e-5, max_iter: int = 100) -> Tuple[float, int]:
    function_calls = 0
    golden_ratio = (3 - np.sqrt(5)) / 2

    x = w = v = a + golden_ratio * (b - a)
    fx = fw = fv = phi(x)
    function_calls += 1

    d = e = 0.0

    for k in range(max_iter):
        xm = 0.5 * (a + b)
        tol1 = tol * abs(x) + 1e-10
        tol2 = 2.0 * tol1

        if abs(x - xm) <= (tol2 - 0.5 * (b - a)):
            return x, function_calls
        
        if abs(e) > tol1:
            r = (x - w) * (fx - fv)
            q = (x - v) * (fx - fw)
            p = (x - v) * q - (x - w) * r
            q = 2.0 * (q - r)

            if q > 0:
                p = -p
            q = abs(q)

            etemp = e
            e = d

            # Kiem tra co chap nhan buoc parabol khong
            parabolic_ok = (abs(p) < abs(0.5 * q * etemp) and p > q * (a - x) and p < q * (b - x))

            if parabolic_ok:
                # Sử dụng bước nội suy parabol
                d = p / q
                u = x + d

                if (u - a) < tol2 or (b - u) < tol2:
                    d = tol1 if xm - x >= 0 else -tol1
            else:
                e = (a - x) if x >= xm else (b - x)
                d = golden_ratio * e
        else:
            e = (a - x) if x >= xm else (b - x)
            d = golden_ratio * e
        
        # Tinh diem moi
        if abs(d) >= tol1:
            u = x + d
        else:
            u = x + (tol1 if d >= 0 else -tol1)
        
        fu = phi(u)
        function_calls += 1

        if fu <= fx:
            if u >= x:
                a = x
            else:
                b = x

            v, w, x = w, x, u
            fv, fw, fx = fw, fx, fu
        else:
            if u < x:
                a = u
            else:
                b = u

            if fu <= fw or w == x:
                v, w = w, u
                fv, fw = fw, fu
            elif fu <= fv or v == x or v == w:
                v = u
                fv = fu
    return x, function_calls

# Thuật toán Gradient Descent

In [76]:
def gradient_descent(method: str, x0: np.ndarray, tol: float = 1e-5, max_iter: int = 1000):
    x = np.array(x0, dtype=float)
    k = 0
    total_calls = 0
    converged = False
    start = time.perf_counter()

    while k < max_iter:
        g = grad_f(x)  # Gradient tại x_k
        if np.linalg.norm(g) < tol:
            converged = True
            break

        # Hàm phi_k(t) = f(x - t * g)
        def phi(t):
            return f(x - t * g)

        # Tìm t_k
        if method == "parabolic":
            t, calls = parabol_line_search(phi, 0.0, 1.0, tol=1e-5)
        elif method == "brent":
            t, calls = brent_line_search(phi, 0.0, 1.0, tol=1e-5)
        else:
            raise ValueError("Unknown method. Use 'parabolic' or 'brent'.")

        total_calls += calls
        x = x - t * g  # Cập nhật x_{k+1}
        k += 1
    end = time.perf_counter()
    total_time = end - start
    print(f"Thời gian thực thi: {total_time: .10}s")
    return x, k, total_calls, converged

In [77]:
print("Phương pháp Parabol")
x_parabolic, iters_parabolic, calls_parabolic, converged_parabolic = gradient_descent("parabolic", x0)
print(f"Nghiệm xấp xỉ: {x_parabolic}")
print(f"Số lần lặp: {iters_parabolic}")
print(f"Tổng số lần gọi hàm f(x): {calls_parabolic}")
print(f"Đã hội tụ: {converged_parabolic}\n")

print("Phương pháp Brent")
x_brent, iters_brent, calls_brent, converged_brent = gradient_descent("brent", x0)
print(f"Nghiệm xấp xỉ: {x_brent}")
print(f"Số lần lặp: {iters_brent}")
print(f"Tổng số lần gọi hàm f(x): {calls_brent}")
print(f"Đã hội tụ: {converged_brent}")

Phương pháp Parabol
Thời gian thực thi:  0.0249512s
Nghiệm xấp xỉ: [-1.0904892 -0.4893139]
Số lần lặp: 87
Tổng số lần gọi hàm f(x): 751
Đã hội tụ: True

Phương pháp Brent
Thời gian thực thi:  0.05482660001s
Nghiệm xấp xỉ: [-1.09048639 -0.48931352]
Số lần lặp: 157
Tổng số lần gọi hàm f(x): 1897
Đã hội tụ: True
